In [1]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import numpy as np
import win32com.client
import os
# VERSION 3.00
import re
#VERSION  3.01
import datetime

""""
- ADDED 'workbook.close()'  AND 'o.Workbooks.Close()' TO MAKE SURE THAT THE MEMORY GETS REUSED 

"""

# VERSION 3.02
"""
- FIXED THE BORDER LINES IN THE EXCEL FILES
- UPDATE ON THE EXCEL TEMPLATE (V2)
"""

'\n- FIXED THE BORDER LINES IN THE EXCEL FILES\n- UPDATE ON THE EXCEL TEMPLATE (V2)\n'

In [2]:
# DATA SOURCE (WHERE YOU ARE GETTING ALL THE DATA SCORES)

                
                #header=[0,1,2],
                   # index_col=None,sheet_name = 'PSH Scoring')
#xl = pd.ExcelFile("N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\\KC_FY2019 Priority List Workbook Final - Full.xlsx")
xl = pd.ExcelFile("N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\\FY2019 Priority List Workbook Final - FULL.xlsx")

pd.set_option('display.max_columns', 500)
xl.sheet_names 

df = xl.parse("TH Scoring",header=[0,1,2])
           

In [3]:
df.head(2)

Source is APR Data filtered to only TH Source is APR Data filtered to only TH  \
**Only APR Grant number Needed                 **Only APR Grant number Needed   
Project #                                                         APR Grant #   
CA0485                                                        CA0485L9D001609   
CA0518                                                        CA0518L9D001609   

Source is APR Data filtered to only TH                            \
**Only APR Grant number Needed                                     
Project #                              Consolidated FileShare ID   
CA0485                                          NaN       LA_FCC   
CA0518                                          NaN       LA_UBH   

Source is APR Data filtered to only TH                             \
**Only APR Grant number Needed                                      
Project #                                                 Grantee   
CA0485                                  1736 Family Crisis Center   
CA0518                                         Upward Bound House   

Source is APR Data filtered to only TH                             \
**Only APR Grant number Needed                                      
Project #                                                  Agency   
CA0485                                  1736 Family Crisis Center   
CA0518                                         Upward Bound House   

Source is APR Data filtered to only TH                                        \
**Only APR Grant number Needed                                                 
Project #                                                            Project   
CA0485                                                    LADV1 Transitional   
CA0518                                  CA0518 Family Place Transitional SHP   

Source is APR Data filtered to only TH                          \
**Only APR Grant number Needed                                   
Project #                                           Population   
CA0485                                       Domestic Violence   
CA0518                                  Families with Children   

Source is APR Data filtered to only TH                                 \
**Only APR Grant number Needed                            Total Score   
Project #                              Cost Effectiveness Prop. Score   
CA0485                                       23120.130435    0.953676   
CA0518                                        3983.125000    1.000000   

Source is APR Data filtered to only TH                                \
**Only APR Grant number Needed                                         
Project #                              Points Earned Points Possible   
CA0485                                     95.367647           100.0   
CA0518                                    100.000000           100.0   

Source is APR Data filtered to only TH Scoring Area 1: HMIS Participation  \
**Only APR Grant number Needed                     Participation (APR Q6)   
Project #                                                        Response   
CA0485                                                                Yes   
CA0518                                                                Yes   

Source is APR Data filtered to only TH                                \
**Only APR Grant number Needed                                         
Project #                              Points Earned Points Possible   
CA0485                                           2.5             2.5   
CA0518                                           2.5             2.5   

Source is APR Data filtered to only TH                                      \
**Only APR Grant number Needed         Data Quality (APR Q7)                 
Project #                                               Rate Points Earned   
CA0485                                                   0.0           2.5   
CA0518                                     

In [4]:
#CHECKING THE GRANTEE NAMES (FOR ANY SIMILARITIES)
#df['Source is APR Data filtered to only TH','**Only APR Grant number Needed','Grantee'].value_counts()

In [4]:
def update_range(worksheet, data, cell_range=None, named_range=None):
     """
     Updates an excel worksheet with the given data.
     :param worksheet: an excel worksheet
     :param data: data used to update the worksheet cell range (list, tuple, np.ndarray, pd.Dataframe)
     :param cell_range: a string representing the cell range, e.g. 'AB12:XX23'
     :param named_range: a string representing an excel named range
     """
 
     def clean_data(data):
         if not isinstance(data, (list, tuple, np.ndarray, pd.DataFrame)):
             raise TypeError('Invalid data, data should be an array type iterable.')
 
         if not len(data):
             raise ValueError('You need to provide data to update the cells')
 
         if isinstance(data, pd.DataFrame):
             data = data.values
 
         elif isinstance(data, (list, tuple)):
             data = np.array(data)
 
         return np.hstack(data)
 
     def clean_cells(worksheet, cell_range, named_range):
         # check that we can access a cell range
         if not any((cell_range, named_range) or all((cell_range, named_range))):
             raise ValueError('`cell_range` or `named_range` should be provided.')
 
         # get the cell range
         if cell_range:
             try:
                 cells = np.hstack(worksheet[cell_range])
             except (CellCoordinatesException, AttributeError):
                 raise ValueError('The cell range provided is invalid, cell range must be in the form XX--[:YY--]')
 
         else:
             try:
                 cells = worksheet.get_named_range(named_range)
             except (NamedRangeException, TypeError):
                 raise ValueError('The current worksheet {} does not contain any named range {}.'.format(
                     worksheet.title,
                     named_range))
 
         # checking that we have cells to update, and data
         if not len(cells):
             raise ValueError('You need to provide cells to update.')
 
         return cells
 
     cells = clean_cells(worksheet, cell_range, named_range)
     data = clean_data(data)
 
     # check that the data has the same dimension as cells
     if len(cells) != data.size:
         raise ValueError('Cells({}) should have the same dimension as the data({}).'.format(len(cells), data.size))
 
     for i, cell in enumerate(cells):
         cell.value = data[i]
            
            


In [6]:
# Division by Zero, but I dont think we will need this function for now. 
def divisionbyzero(numerator,denominator):
    if denominator != 0:
        return numerator/denominator
    else:
        return 0

In [8]:
#CREATING FOLDERS IN THE FILESHARE ID

for index,row in df.iterrows():

    #newpath = r'C:\Users\felixa\Documents\Felix\test'
    k = row['Source is APR Data filtered to only TH','**Only APR Grant number Needed','FileShare ID']
    newpath = r'Q:\\'  + str(k) + '\\Continuum of Care NOFA Renewals\\NOFA 2019\\Renewal Evaluations\\Final Score Reports'
    if not os.path.exists(newpath):
            os.makedirs(newpath)

In [7]:
# EXCEL FILE HAS BEEN UPDATED

In [9]:
for index,row in df.iterrows():
    
    k = row['Source is APR Data filtered to only TH','**Only APR Grant number Needed','APR Grant #']
    GrantN = df[df['Source is APR Data filtered to only TH','**Only APR Grant number Needed','APR Grant #'].str.contains(k, na = False)]
    
    
    """ ----------- REMOVE THIS IF YOU WISH TO UPDATE THE EXCEL FILES--------
    
    #Adding the Values ----------
    VALUES1  = pd.DataFrame(index=['Exit to Permanent Housing', 'Bed/Unit Unitilization', 'Increase Income Overall',
                               'Increase Employment Income', 'Median Length of Stay', 
                                'Commitment to Policy Priorities', 'Spend Down', 'Data Quality', 'HMIS Participation'],
                             
                         columns=['Points Eearned', 'Points Possible'])
    
    
    VALUES1.iloc[0] = GrantN['Scoring Area 4: Performance','Exits to Permanent Housing (Q29)'][['Points Earned','Points Possible']].values
    VALUES1.iloc[1] = GrantN['Scoring Area 2: Bed Utilization','SubTotal'][['Points Earned','Points Possible']].values
    VALUES1.iloc[2] = GrantN['Scoring Area 4: Performance','Increase Income Overall (APR)'][['Points Earned','Points Possible']].values
    
    VALUES1.iloc[3] = GrantN['Scoring Area 4: Performance','Increase Employment Income (APR)'][['Points Earned','Points Possible']].values
    
    VALUES1.iloc[4] = GrantN['Scoring Area 4: Performance','Median Length of Stay Leavers (Q27)'][['Points Earned','Points Possible']].values
    VALUES1.iloc[5] = GrantN['Housing First','SubTotal'][['Points Earned','Points Possible']].values
    VALUES1.iloc[6] = GrantN['Scoring Area 3: Spend-Down','SubTotal'][['Points Earned','Points Possible']].values
    VALUES1.iloc[7] = GrantN['Scoring Area 1: HMIS Participation','Data Quality (APR Q7)'][['Points Earned','Points Possible']].values
    VALUES1.iloc[8] = GrantN['Scoring Area 1: HMIS Participation','Participation (APR Q6)'][['Points Earned','Points Possible']].values
    
    
    VALUES1['APR Result']  = [GrantN['Scoring Area 4: Performance','Exits to Permanent Housing (Q29)']['Rate'][0],
                                  GrantN['Scoring Area 2: Bed Utilization','SubTotal']['Rate'][0],
                                  GrantN['Scoring Area 4: Performance','Increase Income Overall (APR)']['Rate'][0], 
                                  GrantN['Scoring Area 4: Performance','Increase Employment Income (APR)']['Rate'][0],
                                  GrantN['Scoring Area 4: Performance','Median Length of Stay Leavers (Q27)']['Rate'][0],
                                  GrantN['Housing First','Unnamed: 45_level_1']['Response'][0],
                                  GrantN['Scoring Area 3: Spend-Down','Spend-Down Rate (Q31a4)']['Rate'][0],
                                  GrantN['Scoring Area 1: HMIS Participation','Data Quality (APR Q7)']['Rate'][0],
                                  GrantN['Scoring Area 1: HMIS Participation','Participation (APR Q6)']['Response'][0]]

    VALUES1 = VALUES1[['APR Result', 'Points Eearned', 'Points Possible']]
    VALUES1
    
    
    VALUES2 = pd.DataFrame(index = ['Grant Number (APR #)', 'Grantee Name', \
                               'Operating Agency', 'Project Name', 'Project Type', 'Target Population', 'FileShare ID']
                      , columns=['values'])
    
    VALUES2.iloc[0] = GrantN['Source is APR Data filtered to only TH','**Only APR Grant number Needed']['APR Grant #'].values
    VALUES2.iloc[1] = GrantN['Source is APR Data filtered to only TH','**Only APR Grant number Needed']['Grantee'].values
    VALUES2.iloc[2] = GrantN['Source is APR Data filtered to only TH','**Only APR Grant number Needed']['Agency'].values
    VALUES2.iloc[3] = GrantN['Source is APR Data filtered to only TH','**Only APR Grant number Needed']['Project'].values
    # STILL NEEDS TO ADD PROJECT TYPE
    
    VALUES2.iloc[4] = 'TH'#THIS COULD BE A PARAMETER! FOR OUR FUTURE FUNCTION  -FOR THIS CASE WE HAVE RRH 
    VALUES2.iloc[5] = GrantN['Source is APR Data filtered to only TH','**Only APR Grant number Needed']['Population'].values
    
    VALUES2.iloc[6] = GrantN['Source is APR Data filtered to only TH','**Only APR Grant number Needed']['FileShare ID'].values
    
    VALUES2['empty1'] = np.nan
    VALUES2['empty2'] = np.nan
    #VALUES2['empty3'] = np.nan
    
    VALUES2 = VALUES2[['empty1','values','empty2']]
    
    
    VALUES3 = pd.DataFrame(index= ['Score Result'], columns =['Total Points Earned', 'Total Points Possible','Project Final Score'])
    VALUES3.iloc[0] = [VALUES1['Points Eearned'].sum(),VALUES1['Points Possible'].sum() ,VALUES1['Points Eearned'].sum()/VALUES1['Points Possible'].sum() ]
    VALUES3[['Total Points Earned', 'Total Points Possible']]
    
    workbook = load_workbook('N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\TH\Python\Template - TH v2.xlsx')
    worksheet = workbook.active
    worksheet.title = 'template'
    update_range(workbook.active, VALUES1.values, named_range='VALUES1')
    update_range(workbook.active, VALUES2.values, named_range='VALUES2')
    
    
    #total points earned
    update_range(workbook.active, VALUES3[['Total Points Earned', 'Total Points Possible']].values, named_range='VALUES3')


    #Project Final Score
    update_range(workbook.active, VALUES3['Project Final Score'].values, named_range='VALUES4')

    # create a new excel report
    workbook.save(str(k) + '.xlsx')
    
    
    workbook.close() # ADDED
    
    ----DELETE THIS LINE ONLY IF YOU WITH TO UPDATE THE EXCEL FILE--------  """ 
    
    #Turn these into PDF Files ---------
    o = win32com.client.Dispatch("Excel.Application")
    o.Visible = False

    
    
    wb_path =  r'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\TH\Python\\' +  str(k)  + '.xlsx'

    wb = o.Workbooks.Open(wb_path)



    ws_index_list = [1] #say you want to print these sheets

    # PATH TO FILE SHARE (ID) -------------
    
    path_to_pdf = 'Q:\\' + GrantN['Source is APR Data filtered to only TH','**Only APR Grant number Needed']['FileShare ID'] + '\Continuum of Care NOFA Renewals\\NOFA 2019\\Renewal Evaluations\\Final Score Reports\\' + str(k) + '.pdf'
    print(path_to_pdf)
    # PATH TO N DRIVE
    #-------------------------------
    
    # PATH TO THE N DRIVE (BY PROJECT TYPE)
    
    #path_to_pdf = r'N:\Outcomes\CoC Performance Evaluation Process\FY2019 NOFA Performance Evaluation Documents\Priority Listing Working Documents\Evaluation Reports\Final Score Reports\TH\PDF\\' +  str(k)  + '.pdf'
    #print(path_to_pdf) 
    
    
    #wb.WorkSheets(ws_index_list).Select()

    wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)
    
    o.Workbooks.Close() # ADDED 
    
d = datetime.datetime.today()
print("FINAL REPORTS FOR TH HAVE BEEN COMPLETED! :). Date and Time :" ,d.strftime("%d-%B-%Y %H:%M:%S") )      

CA0485    Q:\LA_FCC\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0518    Q:\LA_UBH\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0370    Q:\LA_FCC\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0526    Q:\LA_USV\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0520    Q:\LA_USV\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0482    Q:\LA_COV\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0384    Q:\LA_PAT\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0484    Q:\LA_LAL\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0514    Q:\LA_HOL\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0513    Q:\LA_PLN\Continuum of Care NOFA Renewals\NOFA...
Name: FileShare ID, dtype: object
CA0483    Q:\LA_PLN\Continuum of Care NOFA Renewals\NOFA...
